In [1]:
# train.py (النسخة النهائية والمعدلة)
import tensorflow as tf
import matplotlib.pyplot as plt

# --- الإعدادات الرئيسية للتدريب ---
DATASET_PATH = "C:/Users/ADMIN/Desktop/MyOrganizerProject/data"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10 

print("Loading and preparing dataset...")

# --- 1. تحميل البيانات ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH, validation_split=0.2, subset="training", seed=123,
    image_size=IMAGE_SIZE, batch_size=BATCH_SIZE
)
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH, validation_split=0.2, subset="validation", seed=123,
    image_size=IMAGE_SIZE, batch_size=BATCH_SIZE
)

CLASS_NAMES = train_dataset.class_names
print("-" * 50)
print("IMPORTANT: Classes found by the script:")
print(CLASS_NAMES)
print("Please copy this exact list into the 'CLASS_NAMES' variable in classifier.py")
print("-" * 50)

# --- 2. التعديل الجديد: تطبيق المعالجة المسبقة على البيانات ---
def preprocess_data(image, label):
    # هذه الدالة ستطبق على كل صورة في الداتا ست
    return tf.keras.applications.mobilenet_v2.preprocess_input(image), label

train_dataset = train_dataset.map(preprocess_data).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.map(preprocess_data).prefetch(buffer_size=tf.data.AUTOTUNE)

# --- 3. بناء النموذج (بدون طبقة المعالجة) ---
print("Building the model...")
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet'
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
# x = tf.keras.applications.mobilenet_v2.preprocess_input(x) # <-- تم إزالة هذا السطر
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(x) 
model = tf.keras.Model(inputs, outputs)

# --- 4. تصريف وتدريب النموذج (لا تغيير هنا) ---
print("Compiling and training the model...")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS)

# --- 5. حفظ النموذج (لا تغيير هنا) ---
print("Training complete. Saving the model...")
MODEL_SAVE_PATH = 'my_final_model.h5'
model.save(MODEL_SAVE_PATH)
print(f"Model saved successfully to {MODEL_SAVE_PATH}")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

Loading and preparing dataset...
Found 2498 files belonging to 8 classes.
Using 1999 files for training.
Found 2498 files belonging to 8 classes.
Using 499 files for validation.
--------------------------------------------------
IMPORTANT: Classes found by the script:
['bike', 'cars', 'cats', 'document', 'dogs', 'flowers', 'horses', 'human']
Please copy this exact list into the 'CLASS_NAMES' variable in classifier.py
--------------------------------------------------
Building the model...
Compiling and training the model...
Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 349ms/step - accuracy: 0.6275 - loss: 1.1206 - val_accuracy: 0.9699 - val_loss: 0.1466
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 314ms/step - accuracy: 0.9590 - loss: 0.1548 - val_accuracy: 0.9820 - val_loss: 0.0979
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 315ms/step - accuracy: 0.9534 - loss: 0.1335 - val_accuracy: 0.9820 - val_loss: 0.0812
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 20s 310ms/step - accuracy: 0.9668 - loss: 0.1

Training complete. Saving the model...
Model saved successfully to my_final_model.h5
